# Problem 1 
## Predicting Customer Churn 

This Problem has been solved in the following journal article using SOED: 

[Optimum profit-driven churn decision making: innovative artificial neural networks in telecom industry](classifications[https://link.springer.com/article/10.1007/s00521-020-04850-6)

We will be using kagglehub to import the sample data. Make sure to install it:

`pip install kagglehub`


In [1]:
import kagglehub
import pandas as pd
import os

# Download latest version
path = kagglehub.dataset_download("royjafari/customer-churn")

print("Path to dataset files:", path)

customer_df = pd.read_csv(path+'/Customer Churn.csv')
customer_df

Path to dataset files: /Users/royjafari/.cache/kagglehub/datasets/royjafari/customer-churn/versions/1


,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,FN,FP,Churn
0,8,0,38,0,4370,71,5,17,3,1,1,30,197.640,177.8760,69.7640,0
1,0,0,39,0,318,5,7,4,2,1,2,25,46.035,41.4315,60.0000,0
2,10,0,37,0,2453,60,359,24,3,1,1,30,1536.520,1382.8680,203.6520,0
3,10,0,38,0,4198,66,1,35,1,1,1,15,240.020,216.0180,74.0020,0
4,3,0,38,0,2393,58,2,33,1,1,1,15,145.805,131.2245,64.5805,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,21,0,19,2,6697,147,92,44,2,2,1,25,721.980,649.7820,122.1980,0
3146,17,0,17,1,9237,177,80,42,5,1,1,55,261.210,235.0890,76.1210,0
3147,13,0,18,4,3157,51,38,21,3,1,1,30,280.320,252.2880,78.0320,0
3148,7,0,11,2,4695,46,222,12,3,1,1,30,1077.640,969.8760,157.7640,0


In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from soed import SOEDClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

In [3]:
customer_df.columns

Index(['Call  Failure', 'Complains', 'Subscription  Length', 'Charge  Amount',
       'Seconds of Use', 'Frequency of use', 'Frequency of SMS',
       'Distinct Called Numbers', 'Age Group', 'Tariff Plan', 'Status', 'Age',
       'Customer Value', 'FN', 'FP', 'Churn'],
      dtype='object')

In [4]:
customer_df['Customer Value'] = customer_df['Customer Value'].replace({0:1}).sort_values()

In [5]:
X = customer_df[['Call  Failure', 'Complains', 'Subscription  Length', 'Charge  Amount',
       'Seconds of Use', 'Frequency of use', 'Frequency of SMS',
       'Distinct Called Numbers', 'Age Group', 'Tariff Plan', 'Status', 'Age',
       'Customer Value']]
y = customer_df['Churn']
c = customer_df[['FN','FP']]

c.loc[:,:] = np.where(c==0.0,1,c) 

c.loc[:,'FN'] = np.where(y==0,0,X['Customer Value'])
c.loc[:,'FP'] = np.where(y==1,0,2.0)
c.join(y)

/var/folders/xc/x5p7472d67ddgmb6b_5fhgm80000gn/T/ipykernel_72056/2303209761.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c.loc[:,:] = np.where(c==0.0,1,c)


,FN,FP,Churn
0,0.00,2.0,0
1,0.00,2.0,0
2,0.00,2.0,0
3,0.00,2.0,0
4,0.00,2.0,0
...,...,...,...
3145,0.00,2.0,0
3146,0.00,2.0,0
3147,0.00,2.0,0
3148,0.00,2.0,0


### Standardizing Data

In [6]:
X  = (X - X.mean())/X.std()

In [7]:
X.describe()

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value
count,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03
mean,-5.413659e-17,-3.834675e-17,1.939894e-16,-6.315935e-17,-1.218073e-16,8.346058e-17,-1.691768e-17,-6.541505e-17,-9.022765e-17,1.951173e-16,2.086514e-16,1.624098e-16,1.860945e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.050118e+00,-2.877847e-01,-3.445730e+00,-6.198636e-01,-1.065402e+00,-1.209835e+00,-6.519646e-01,-1.365475e+00,-2.045848e+00,-2.903628e-01,-5.745708e-01,-1.811600e+00,-9.091595e-01
25%,-9.124506e-01,-2.877847e-01,-2.964845e-01,-6.198636e-01,-7.339868e-01,-7.395609e-01,-5.985066e-01,-7.846650e-01,-9.254686e-01,-2.903628e-01,-5.745708e-01,-6.792377e-01,-6.909657e-01
50%,-2.241137e-01,-2.877847e-01,2.867091e-01,-6.198636e-01,-3.531424e-01,-2.692866e-01,-4.648615e-01,-1.457741e-01,1.949104e-01,-2.903628e-01,-5.745708e-01,-1.130565e-01,-4.691402e-01
75%,6.018905e-01,-2.877847e-01,6.366253e-01,3.756749e-02,4.778070e-01,4.448335e-01,1.231769e-01,6.092788e-01,1.949104e-01,-2.903628e-01,-5.745708e-01,-1.130565e-01,6.139030e-01
max,3.905907e+00,3.473717e+00,1.686374e+00,5.954448e+00,3.005673e+00,3.231644e+00,3.998885e+00,4.268381e+00,2.435668e+00,3.442874e+00,1.739877e+00,2.717849e+00,3.277253e+00


### PCA Transformation

In [8]:
pca = PCA(n_components=7)  # Reduce to 7 dimensions
X_pca = pd.DataFrame(pca.fit_transform(X),columns = [f'PC{i}' for i in range(1,8)])

In [9]:
X_pca.describe()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7
count,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03,3.150000e+03
mean,-5.413659e-17,7.218212e-17,-2.255691e-18,-4.060244e-17,2.706829e-17,-1.804553e-17,-4.962521e-17
std,2.004368e+00,1.522979e+00,1.280825e+00,1.103715e+00,1.087492e+00,9.220586e-01,7.588744e-01
min,-3.324694e+00,-3.121313e+00,-3.970316e+00,-3.747483e+00,-1.922499e+00,-3.716016e+00,-2.528156e+00
25%,-1.556205e+00,-9.243099e-01,-7.732543e-01,-5.448963e-01,-7.442458e-01,-6.039511e-01,-5.565888e-01
50%,-2.405729e-01,3.754805e-02,-1.778013e-01,2.158259e-01,-1.682587e-01,-1.410513e-01,-2.695039e-02
75%,1.271029e+00,6.473177e-01,8.876665e-01,7.504528e-01,2.748549e-01,5.051470e-01,5.707165e-01
max,7.562748e+00,5.576084e+00,4.404363e+00,2.305983e+00,5.351844e+00,3.369469e+00,2.588019e+00


### Split Data

In [10]:
random_index = np.random.permutation(X.shape[0])

i = int(round(X.shape[0]*0.5))
train_index = random_index[:i]
test_index = random_index[i+1:]

X_train = X_pca.iloc[train_index]
X_test = X_pca.iloc[test_index]

y_train = y[train_index]
y_test = y[test_index]

In [11]:
X_train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7
3093,-0.245555,0.328908,-0.128921,-0.812748,-0.630052,0.105253,-0.452002
1253,-0.542220,-2.018506,-1.664526,-0.464898,-1.093912,-0.157165,-0.611194
2726,-1.322519,0.544881,-0.898977,0.826337,0.235605,-0.631178,0.527818
687,0.237990,-1.744112,0.457235,0.631348,-0.195197,0.058215,-0.451815
2729,-1.468191,0.112818,0.100149,0.053450,-0.539314,-0.527206,-0.541600
...,...,...,...,...,...,...,...
972,-1.087301,-0.181725,-1.872859,0.610222,0.614404,-0.823143,-0.112010
3010,-0.332963,0.177217,-0.254833,0.331450,-1.022846,-0.158115,0.095636
2272,-0.732139,1.225587,-2.445131,1.854673,3.105299,1.379765,-0.403635
546,0.880198,1.625355,0.770302,-1.490612,-1.079636,1.693546,0.913178


### Model Trainig

In [12]:
soed = SOEDClassifier(mlp_max_iter=10000,som_x=20,som_y=20,som_sigma=20,som_input_len=X_train.shape[1])
soed.fit(X_train.values,y_train)

Model training complete.


### Model Evaluation

In [13]:
y_proba = soed.predict_proba(X_test.values)
y_pred = soed.predict(X_test.values)

recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
auc = roc_auc_score(y_test,y_proba[:,1])

performance = {'recall':recall,'precision':precision,'accuracy':accuracy,'auc':auc}

In [14]:
print(performance)

{'recall': 0.5333333333333333, 'precision': 0.9066666666666666, 'accuracy': 0.9155019059720457, 'auc': np.float64(0.9300703147066255)}


### Model Comparison

In [15]:
report_df = pd.DataFrame(index = [f'repeat{i}' for i in range(1,6)],
                         columns = ['DT','KNN','MLP','SOED'] )
report_df

,DT,KNN,MLP,SOED
repeat1,NaN,NaN,NaN,NaN
repeat2,NaN,NaN,NaN,NaN
repeat3,NaN,NaN,NaN,NaN
repeat4,NaN,NaN,NaN,NaN
repeat5,NaN,NaN,NaN,NaN


In [16]:
for loop_i in range(1,6):
    print(loop_i)
    random_index = np.random.permutation(X.shape[0])

    i = int(round(X.shape[0]*0.5))
    train_index = random_index[:i]
    test_index = random_index[i+1:]
    
    X_train = X_pca.iloc[train_index]
    X_test = X_pca.iloc[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

    #soed
    soed = SOEDClassifier(mlp_max_iter=10000,som_x=15,som_y=15,som_sigma=20,som_input_len=X_train.shape[1])
    soed.fit(X_train.values,y_train)
    y_proba = soed.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','SOED'] = auc

    #dt
    dt = DecisionTreeClassifier(max_depth=10)
    dt.fit(X_train.values,y_train)
    y_proba = dt.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','DT'] = auc

    #mlp
    mlp = MLPClassifier(max_iter=10000)
    mlp.fit(X_train.values,y_train)
    y_proba = mlp.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','MLP'] = auc

    #knn
    knn = KNeighborsClassifier()
    knn.fit(X_train.values,y_train)
    y_proba = knn.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','KNN'] = auc
    

report_df.loc['Average'] = report_df.mean()

1
Model training complete.
2
Model training complete.
3
Model training complete.
4
Model training complete.
5
Model training complete.


In [17]:
report_df

,DT,KNN,MLP,SOED
repeat1,0.859919,0.950213,0.967834,0.915539
repeat2,0.846729,0.949117,0.977999,0.948904
repeat3,0.912832,0.950178,0.978236,0.923797
repeat4,0.88043,0.952076,0.975741,0.949954
repeat5,0.89226,0.966664,0.975587,0.93305
Average,0.878434,0.95365,0.975079,0.934249


We can see that SOED is performing worse than MLP and KNN. SOED will show its superiority in the decision-making problem. 

# Problem 2
## Deciding Customer Loyalty actions

The FN and FP columns in this dataset are calculated based on the actual customers, making this a perfect example for the SEOD algorithm. Let's use it.


In [18]:
random_index = np.random.permutation(X.shape[0])

i = int(round(X.shape[0]*0.5))
train_index = random_index[:i]
test_index = random_index[i+1:]

X_train = X_pca.iloc[train_index]
X_test = X_pca.iloc[test_index]

y_train = y[train_index]
y_test = y[test_index]

c_train = c.loc[train_index,:]
c_test = c.loc[test_index,:]

In [19]:
soed = SOEDClassifier(mlp_max_iter=10000,som_x=20,som_y=20,som_sigma=20,som_input_len=X_train.shape[1])
soed.fit(X_train.values,y_train,c_train.values)

Model training complete.


In [20]:
y_decide = soed.decide(X_test.values)

In [21]:
df = pd.DataFrame(np.column_stack((y_test,y_decide,c_test)),columns = ['Reality','Decision','FN','FP'])
df['FN_cost'] = np.where((df.Decision==0) & (df.Reality==1),df.FN,0)
df['FP_cost'] = np.where((df.Decision==1) & (df.Reality==0),df.FP,0)
df['cost'] = df.FP_cost + df.FN_cost
df.sort_values('cost')

,Reality,Decision,FN,FP,FN_cost,FP_cost,cost
0,1.0,1.0,175.160,0.0,0.000,0.0,0.000
1014,0.0,0.0,0.000,2.0,0.000,0.0,0.000
1013,0.0,0.0,0.000,2.0,0.000,0.0,0.000
1010,0.0,0.0,0.000,2.0,0.000,0.0,0.000
1009,0.0,0.0,0.000,2.0,0.000,0.0,0.000
...,...,...,...,...,...,...,...
157,1.0,0.0,288.630,0.0,288.630,0.0,288.630
896,1.0,0.0,306.765,0.0,306.765,0.0,306.765
945,1.0,0.0,330.520,0.0,330.520,0.0,330.520
648,1.0,0.0,349.065,0.0,349.065,0.0,349.065


In [22]:
total_cost = df.cost.sum()
print(f'total cost is {total_cost}.')

total cost is 4210.915000000001.


### Comparing Models

In [23]:
report_df = pd.DataFrame(index = [f'repeat{i}' for i in range(1,6)],
                         columns = ['DT','MLP','SOED'] )
report_df

,DT,MLP,SOED
repeat1,NaN,NaN,NaN
repeat2,NaN,NaN,NaN
repeat3,NaN,NaN,NaN
repeat4,NaN,NaN,NaN
repeat5,NaN,NaN,NaN


In [24]:
def calc_cost(y_reality,y_decide,c):
    #print(y_reality.shape, y_decide.shape, c.shape)
    df = pd.DataFrame(np.column_stack((y_reality,y_decide,c)),columns = ['Reality','Decision','FN','FP'])
    df['FN_cost'] = np.where((df.Decision==0) & (df.Reality==1),df.FN,0)
    df['FP_cost'] = np.where((df.Decision==1) & (df.Reality==0),df.FP,0)
    df['cost'] = df.FP_cost + df.FN_cost
    return df.cost.sum()

In [25]:
def get_cost_minimizing_threshold(y_reality,y_prob,c):
    candidate_df = pd.DataFrame(index=range(99),columns = ['thresh','cost'])
    candidate_df.thresh = np.linspace(0.01,0.99,99)
    candidate_df = candidate_df.set_index('thresh')
    
    for t in candidate_df.index.tolist():
        y_decide = np.where(y_prob>t,1,0)
        candidate_df.loc[t,'cost'] = calc_cost(y_reality,y_decide,c)
    return candidate_df[candidate_df.cost == candidate_df.cost.min()].index[0]

In [26]:
for loop_i in range(1,6):
    print(loop_i)
    random_index = np.random.permutation(X.shape[0])

    i = int(round(X.shape[0]*0.7))
    train_index = random_index[:i]
    test_index = random_index[i+1:]
    
    X_train = X_pca.iloc[train_index]
    X_test = X_pca.iloc[test_index]
    
    y_train = y[train_index].values
    y_test = y[test_index].values

    c_train = c.loc[train_index,:].values
    c_test = c.loc[test_index,:].values

    #soed
    soed = SOEDClassifier(mlp_max_iter=10000,som_x=20,som_y=20,som_sigma=20,som_input_len=X_train.shape[1])
    soed.fit(X_train.values,y_train,c_train)
    y_decide = soed.decide(X_test.values)
    total_cost= calc_cost(y_test,y_decide,c_test)
    report_df.loc[f'repeat{loop_i}','SOED'] = total_cost

    #dt
    dt = DecisionTreeClassifier()
    dt.fit(X_train.values,y_train)

    y_prob = dt.predict_proba(X_train.values)[:,1]
    thresh = get_cost_minimizing_threshold(y_train,y_prob,c_train)
    print(f'dt threshod= {thresh}')
    y_prob = dt.predict_proba(X_test.values)[:,1]
    y_decide = np.where(y_prob>thresh,1,0)
    total_cost= calc_cost(y_test,y_decide,c_test)
    report_df.loc[f'repeat{loop_i}','DT'] = total_cost

    #mlp
    mlp = MLPClassifier(max_iter=10000)
    mlp.fit(X_train.values,y_train)
    
    y_prob = mlp.predict_proba(X_train.values)[:,1]
    thresh = get_cost_minimizing_threshold(y_train,y_prob,c_train)
    print(f'mlp threshod= {thresh}')
    y_prob = mlp.predict_proba(X_test.values)[:,1]
    y_decide = np.where(y_prob>thresh,1,0)
    total_cost= calc_cost(y_test,y_decide,c_test)
    report_df.loc[f'repeat{loop_i}','MLP'] = total_cost


report_df.loc['Average'] = report_df.mean()

1
Model training complete.
dt threshod= 0.2
mlp threshod= 0.08
2
Model training complete.
dt threshod= 0.25
mlp threshod= 0.03
3
Model training complete.
dt threshod= 0.34
mlp threshod= 0.02
4
Model training complete.
dt threshod= 0.34
mlp threshod= 0.05
5
Model training complete.
dt threshod= 0.01
mlp threshod= 0.02


In [27]:
report_df

,DT,MLP,SOED
repeat1,6421.355,1317.53,2702.66
repeat2,5600.405,943.23,4021.155
repeat3,5649.165,657.325,3862.485
repeat4,5929.97,1714.76,3656.19
repeat5,4527.62,558.15,1755.625
Average,5625.703,1038.199,3199.623


In this example, MLP is superior to SOED.